Read extrated data from the input file

In [2]:
import numpy as np
import math
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn import decomposition
from sklearn import svm
from sklearn import decomposition
import csv

In [3]:
result=np.array(list(csv.reader(open("train_1.csv","rb"),delimiter=',')))

Remove first three columns as these are for names etc.

In [4]:
X = result[:,3:, ]

In [5]:
X = X.astype('float64')

Remove Nans from the data.

In [6]:
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        if(math.isnan(X[i,j])):
            X[i,j] = 0.0

 Do PCA on set of extracted features from the raw EEG data

In [69]:
from sklearn.cross_validation import train_test_split 

y = np.array(result[:,2])

X_train, X_test, y_train, y_test = train_test_split(X,y)

pca = decomposition.PCA()
pca.n_components = 620
train_reduced = pca.fit_transform(X_train)
test_reduced = pca.transform(X_test)

X_train = train_reduced
X_test = test_reduced

1 . Multilayer perceptron with relu as activation function

In [52]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='sgd', hidden_layer_sizes=(119, 35, 14 ,2), random_state=1, activation='relu', max_iter=20000)
clf.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(119, 35, 14, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=20000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='sgd', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [53]:
from sklearn.metrics import confusion_matrix
cnf_matrix = confusion_matrix(y_test, clf.predict(X_test))
print(cnf_matrix)

[[290   0]
 [ 36   0]]


2 . No luck with above MLP, lets try with tanh as activation function

In [54]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='sgd', hidden_layer_sizes=(119, 35, 14 ,2), random_state=1, activation='tanh', max_iter=20000)
clf.fit(X_train, y_train)

MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(119, 35, 14, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=20000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='sgd', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [55]:
from sklearn.metrics import confusion_matrix
cnf_matrix = confusion_matrix(y_test, clf.predict(X_test))
print(cnf_matrix)

[[285   5]
 [ 36   0]]


3 . No help with the tanh activation fnction either,
Try other classifier, ensemble learning might help better

In [74]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=135)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=135, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [75]:
cnf_matrix = confusion_matrix(y_test, rf.predict(X_test))
print(cnf_matrix)

[[226  74]
 [  9  17]]


3 . Slightly better than other (with n_estimator = 135 and number of components 620 in PCA), lets try the adaboost 

In [76]:
from sklearn.ensemble import AdaBoostClassifier
ab = AdaBoostClassifier(n_estimators=100, learning_rate=0.01)
ab.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=0.01, n_estimators=100, random_state=None)

In [77]:
cnf_matrix = confusion_matrix(y_test, ab.predict(X_test))
print(cnf_matrix)

[[300   0]
 [ 26   0]]


4 . We back to where started, lets try for GradientBoosting

In [78]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.01, max_depth=9)
gb.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.01, loss='deviance', max_depth=9,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)

In [79]:
cnf_matrix = confusion_matrix(y_test, gb.predict(X_test))
print(cnf_matrix)

[[277  23]
 [ 22   4]]
